# Training a Convolutional Neural Network for a computer vision binary classification task

From https://github.com/dsikar/city-dss-cnn-workshop-1


In [ ]:
# get the data
# Install PyDrive
!pip install PyDrive

#Import modules
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Get the shareable link e.g. https://drive.google.com/file/d/19sU5tk8RQY605GGjPJMW8mx3S9HrKaFW/view?usp=sharing
# Get the id from the link 1JYdxKasFKNfVqx2Iklj7iv_bwPanmi4j
downloaded = drive.CreateFile({'id':"19sU5tk8RQY605GGjPJMW8mx3S9HrKaFW"})   
# Remember at least what was the file extension
downloaded.GetContentFile('datasets.tar.gz')       

In [ ]:
# list - show manual for ls command: man ls 
!ls

In [ ]:
# decompress
# man tar - NB exclamation mark is a google colab directive
!tar xvf datasets.tar.gz

In [ ]:
# list drive contents
!ls -lh datasets
!ls -l datasets/fire | wc -l
!ls -l datasets/nofire | wc -l
# note, the dataset is **unbalanced**, more "fire" than "no fire" examples. Briefly consider self-driving car data
# with front camera images mainly taken when driving straight. Fraud detection problem, etc.

In [ ]:
# set data location and categories/labels
DATADIR = 'datasets/'
CATEGORIES = ['nofire', 'fire']
print(type(DATADIR))
print(type(CATEGORIES))
print(CATEGORIES[0])
print(CATEGORIES[1])
print(type(CATEGORIES[1]))

In [ ]:
# import libraries
import os
import cv2
import numpy as np
from tqdm import tqdm
# cv2_imshow - google colab substitute to display images
from google.colab.patches import cv2_imshow
# plot a couple of images
plot_size = 200
img_fire = cv2.imread('datasets/fire/20200724_175212_001.jpg')
img_fire = cv2.resize(img_fire, (plot_size, plot_size))
print("FIRE")
cv2_imshow(img_fire)
img_no_fire = cv2.imread('datasets/nofire/20200724_175219_001.jpg')
print("NO FIRE")
img_no_fire = cv2.resize(img_no_fire, (plot_size, plot_size))
cv2_imshow(img_no_fire)
print("\n\rimg_no_fire data type:", type(img_no_fire))

In [ ]:
# define function
def create_training_data(IMG_SIZE, DATADIR, CATEGORIES):
  """
  Create a training dataset
  Arguments:
    IMG_SIZE: int, square side size images will be resized to
    DATADIR: path to data
    CATEGORIES: list, list of classification categories AKA lables
  Returns:
    training_data: list with training data and categories/labels
  """
  training_data = []
  for category in CATEGORIES:  

      path = os.path.join(DATADIR,category) # this looks like dataset/fire for index 0
      class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=fire, 1=no fire

      for img in tqdm(os.listdir(path)):  # iterate over each image
          try:
              img_array = cv2.imread(os.path.join(path,img))  # convert to array
              new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
              training_data.append([new_array, class_num])  # add array of image array and label to list
          except Exception as e:  # todo exception handling
              pass
            
  return training_data

In [ ]:
# define image size for training data
IMG_SIZE = 64 # 64 x 64 pixels
training_data = create_training_data(IMG_SIZE, DATADIR, CATEGORIES)

In [ ]:
# Now we have a list containing image arrays and labels,
# let's convince ourselves that the data is in training_data, with the labels
# Print first label in training_labels list.
print(training_data[0][1]) # the zeroeth label and image
cv2_imshow(training_data[0][0])
print(training_data[200][1]) # the 200th label and image
cv2_imshow(training_data[200][0])

In [ ]:
X = []
Y = []
# separate elemements of each (image, label) tuple
for features,label in training_data: 
    X.append(features)
    Y.append(label)

Y = np.asarray(Y)
print(Y.shape)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0 # normalise pixel values - another big topic we'll not cover for now
X.shape[1:] # the input shape of our neural network - VERY IMPORTANT
            # all pre-processing must be done to image at "inference" time

In [ ]:
# Data augmentation - Another important topic we will not cover this time
# set up image augmentation
#from keras.preprocessing.image import ImageDataGenerator

#datagen = ImageDataGenerator(
#     rotation_range=15,
#     horizontal_flip=True,
#     width_shift_range=0.1,
#     height_shift_range=0.1
#     #zoom_range=0.3
#     )
#Xaug = datagen.fit(X)

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

model = Sequential()


model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=2, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=100,validation_split=0.3)
# model.fit_generator(datagen.flow(X, Y, batch_size=32),
#                     epochs=100,
#                     verbose=1)

#history = model.fit(X, Y, batch_size=27, epochs=10,validation_split=0.3)

In [ ]:
print(type(history))
print(history.history.keys())
history.history['loss']

In [ ]:
# save model
model.save('firemodel.h5')

In [ ]:
# verify 
!ls -lh firemodel.h5

In [ ]:
# plot training accuracy
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# plot training loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# Using the model in practice
# load model (for future reference as it is already in memory)
import tensorflow as tf
from tensorflow.keras.utils import plot_model
# model = tf.keras.models.load_model('TrainedModels/Fire-64x64-color-v7-soft.h5')
model = tf.keras.models.load_model('firemodel.h5')
# TODO, download model and run locally

In [ ]:
# single prediction fire/no fire
plot_size = 64
img = cv2.imread('datasets/fire/20200724_175212_001.jpg')
# image must go through same transformations
img = cv2.resize(img, (plot_size, plot_size))
# plot
cv2_imshow(img)
img = img.astype("float32") / 255.0
pred = model.predict(np.expand_dims(img, axis=0))[0]
print(pred)
print("Prediction of no fire:", pred[0])
print("Prediction of fire:", pred[1])
img = cv2.imread('datasets/nofire/20200724_175219_001.jpg')
# image must go through same transformations
img = cv2.resize(img, (plot_size, plot_size))
cv2_imshow(img)
img = img.astype("float32") / 255.0
pred = model.predict(np.expand_dims(img, axis=0))[0]
print(pred)
print("Prediction of no fire:", pred[0])
print("Prediction of fire:", pred[1])
# Takeaways
# The training data must be known in order
# to interpret results